In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
import torch.nn as nn
import wandb
import torch
random_seed = 42

In [134]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
#train = pd.read_csv('json_train.csv')
#train = train.drop("Unnamed: 0", axis = 1)
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)


In [135]:
X_train

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
7676,675.0,22.0,7.0,1.0,0.0,74.0,0.0,1.0,0.0,10.0,...,12.997,0.0,0.0,2.0,0.0,0.667,0.0,0.0,0.0,0.0
1257,203.0,22.0,7.0,1.0,7.0,19.0,0.0,0.0,0.0,4.0,...,2.433,0.0,0.0,1.0,0.0,0.167,0.0,0.0,1.0,0.0
24896,1129.0,22.0,7.0,4.0,10.0,5.0,2.0,7.0,6.0,5.0,...,11.331,0.0,0.0,4.0,0.0,0.312,1.0,0.0,0.0,0.0
7056,1768.0,22.0,7.0,8.0,7.0,120.0,1.0,4.0,6.0,11.0,...,0.000,3.0,2.0,3.0,0.0,0.400,0.0,1.0,0.0,0.0
21579,1489.0,22.0,7.0,8.0,10.0,88.0,2.0,5.0,2.0,4.0,...,10.631,15.0,4.0,8.0,0.0,0.409,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23444,2434.0,22.0,0.0,16.0,0.0,91.0,2.0,10.0,20.0,0.0,...,1.733,0.0,0.0,2.0,0.0,0.591,0.0,0.0,9.0,2.0
9392,1040.0,22.0,7.0,4.0,12.0,12.0,8.0,4.0,7.0,6.0,...,7.598,0.0,0.0,0.0,0.0,0.158,0.0,0.0,0.0,0.0
1499,2102.0,22.0,7.0,10.0,23.0,41.0,4.0,12.0,3.0,2.0,...,0.000,0.0,0.0,6.0,1.0,0.500,0.0,0.0,11.0,14.0
27569,1148.0,22.0,7.0,4.0,4.0,62.0,1.0,7.0,7.0,0.0,...,0.000,3.0,1.0,4.0,0.0,0.188,0.0,0.0,2.0,3.0


In [136]:
train_dataset = ut.MyTensorDataset(torch.FloatTensor(X_train.values), torch.FloatTensor(Y_train.values))
test_dataset = ut.MyTensorDataset(torch.FloatTensor(X_test.values), torch.FloatTensor(Y_test.values))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=False,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False,)

In [137]:
size = len(X_train.columns)
net = nn.Sequential(nn.Linear(size, size), nn.Linear(size, 1), nn.Sigmoid())
n_epoch = 20
criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum= 0.1, weight_decay=10)

In [138]:
size

245

In [139]:
net[0].weight

Parameter containing:
tensor([[-0.0425,  0.0208, -0.0067,  ..., -0.0369, -0.0272,  0.0267],
        [ 0.0380,  0.0247, -0.0097,  ...,  0.0353,  0.0184, -0.0376],
        [-0.0230,  0.0023,  0.0370,  ..., -0.0617,  0.0320, -0.0370],
        ...,
        [-0.0596,  0.0507,  0.0105,  ..., -0.0363,  0.0148, -0.0388],
        [-0.0490,  0.0389, -0.0237,  ...,  0.0345,  0.0423,  0.0278],
        [-0.0535,  0.0552, -0.0018,  ...,  0.0293, -0.0295, -0.0058]],
       requires_grad=True)

In [140]:
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in train_dataloader:
        #print(x_batch)
        #y_batch = torch.transpose(y_batch, 0, 1)[0]
        #y_batch = y_batch[0]
        #print(y_batch)
        # Forward
        predictions = net(x_batch)
        #print(predictions)
        loss = criterion(predictions, y_batch)

        # Backward
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        loss_history.append(loss.item())
    print(f'{i+1}/{n_epoch} current loss: {loss}')

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


1/20 current loss: 1.31326162815094
2/20 current loss: 1.31326162815094
3/20 current loss: 1.31326162815094
4/20 current loss: 1.31326162815094
5/20 current loss: 1.31326162815094
6/20 current loss: 1.31326162815094
7/20 current loss: 1.31326162815094
8/20 current loss: 1.31326162815094
9/20 current loss: 1.31326162815094
10/20 current loss: 1.31326162815094
11/20 current loss: 1.31326162815094
12/20 current loss: 1.31326162815094
13/20 current loss: 1.31326162815094
14/20 current loss: 1.31326162815094
15/20 current loss: 1.31326162815094
16/20 current loss: 1.31326162815094
17/20 current loss: 1.31326162815094
18/20 current loss: 1.31326162815094
19/20 current loss: 1.31326162815094
20/20 current loss: 1.31326162815094


In [141]:
test_preds = np.array([])

for x_batch, y_batch in test_dataloader:
    predictions = net(x_batch)
    predictions = torch.transpose(predictions, 0, 1)[0]
    predictions = predictions.detach().numpy()
    test_preds = np.append(test_preds, predictions)

In [142]:
test_preds 

array([1., 1., 1., ..., 1., 1., 1.])

In [143]:
from sklearn.metrics import roc_auc_score
#print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, test_preds)}")

ROC-AUC score : 0.5


In [144]:
ut.writeNN(net)

c:\Users\Oleg\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
